# Загрузка Pandas и очистка данных

In [559]:
import pandas as pd
import json

pd.set_option("max_rows",10)

# Restaurant_id — идентификационный номер ресторана / сети ресторанов;
# City — город, в котором находится ресторан;
# Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
# Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
# Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
# Price Range — диапазон цен в ресторане;
# Number of Reviews — количество отзывов о ресторане;
# Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
# URL_TA — URL страницы ресторана на TripAdvosor;
# ID_TA — идентификатор ресторана в базе данных TripAdvisor.

In [560]:
original_df = pd.read_csv('main_task.csv')
new_column_names = {}
for column in original_df.columns:
    new_column_name = column.lower()
    new_column_name = new_column_name.replace(' ','_')
    new_column_names[column] = new_column_name.strip()
new_column_names['Number of Reviews'] = 'reviews_number'
original_df.rename(new_column_names,axis=1,inplace=True)

original_df.head(5)


,restaurant_id,city,cuisine_style,ranking,rating,price_range,reviews_number,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [561]:
cuisine_styles = pd.DataFrame({'id_ta':original_df['id_ta'], 'cuisine_style':original_df['cuisine_style']})
cuisine_styles['cuisine_style'] = cuisine_styles['cuisine_style'].apply(lambda x: x.strip('[]').replace("'","").replace(" ","").split(',') if type(x) is str else ['none'])
cuisine_styles = cuisine_styles.set_index(['id_ta'])
cuisine_styles = (cuisine_styles.apply(lambda x: pd.DataFrame(x.tolist(),index=x.index)
                        .stack()
                        .rename(x.name)).reset_index())

In [562]:
#4.1.1 Сколько вариантов непустых значений встречается в столбце Price Range? (3)
#4.1.2 Как в датафрейме обозначается самый низкий уровень цен? ($)
#4.1.3 Как в датафрейме обозначается самый высокий уровень цен? ($$$$)
#4.1.4 Сколько ресторанов относятся к среднему ценовому сегменту? (18412)
print(original_df['price_range'].value_counts())

$$ - $$$    18412
$            6279
$$$$         1423
Name: price_range, dtype: int64


In [563]:
# 4.2 Сколько городов представлено в наборе данных? (31)?
original_df['city'].value_counts()

London        5757
Paris         4897
Madrid        3108
Barcelona     2734
Berlin        2155
              ... 
Oslo           385
Helsinki       376
Bratislava     301
Luxembourg     210
Ljubljana      183
Name: city, Length: 31, dtype: int64

In [564]:
# 4.3.1 Сколько типов кухонь представлено в наборе данных? (125)
# 4.3.2 Какая кухня представлена в наибольшем количестве ресторанов? (Vegetarian Friendly)
cuisine_styles.groupby('cuisine_style')['id_ta'].count().sort_values(ascending=False)

cuisine_style
VegetarianFriendly    11189
European              10060
none                   9283
Mediterranean          6277
Italian                5964
                      ...  
Xinjiang                  1
Latvian                   1
Salvadoran                1
Yunnan                    1
Burmese                   1
Name: id_ta, Length: 126, dtype: int64

In [565]:
# 4.3.3 Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует
# информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. (2.6)
cuisine_styles.groupby('id_ta')['cuisine_style'].count().sort_values(ascending=False).mean()

2.623711855927964

In [566]:
#4.4 Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.

# res = json.load(original_df.iloc[0]['reviews'])

# original_df.iloc[0]['reviews']
# res = json.loads('["Good food at your doorstep", "A good hotel restaurant"]')

blackjack_hand = (8, "Q")
# encoded_hand = json.dumps(blackjack_hand)
encoded_hand = json.dumps('{}')
decoded_hand = json.loads(encoded_hand)
print(encoded_hand)
print(type(decoded_hand))

# decoded_hand = json.loads(encoded_hand)
# print(encoded_hand)
# print(decoded_hand)
# print(json.loads("[8, 'Q']"))

"{}"
<class 'str'>


In [567]:
# Ваш код по очистке данных и генерации новых признаков
numeric = ['ranking','rating','reviews_number']
for column in original_df.columns:
    if column not in numeric:
        original_df.drop(column,inplace=True,axis=1)

original_df['ranking'].fillna(0,inplace=True)
original_df['rating'].fillna(0,inplace=True)
original_df['reviews_number'].fillna(0,inplace=True)
original_df.head()

,ranking,rating,reviews_number
0,5570.0,3.5,194.0
1,1537.0,4.0,10.0
2,353.0,4.5,688.0
3,3458.0,5.0,3.0
4,621.0,4.0,84.0


In [568]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
#X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
X = original_df.drop(['rating'], axis = 1)
y = original_df['rating']

In [569]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [570]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [571]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [572]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [573]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))


MAE: 0.4267376073412699
